# Quick exercise: Munk balance

a) Take $Q_s=15 \times 10^6 m^3/s$ and $A=4 \times 10^{13} m^2$. What is $w$?

b) Run all the cells in the notebook. Find where the exponential function is defined and write down the parameters that the code spits out

c) What is the parameter c in the code? How does it relate to $K_v$ and $w$?

d) Calculate $K_v$ based on $c$ and $w$


In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import fsspec

url = "https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/nodc.woa98/temperat/annual/otemp.raw1deg.nc"


with fsspec.open(url) as fp:
    ds = xr.open_dataset(fp,decode_times=False).load()
ds

In [ ]:
#find the mean temperature profile over a small region
T_profile = ds.otemp[0].sel(lon=slice(160, 170)).isel(lat=70).mean('lon')
T_profile

In [ ]:
# Fit a function to the data
from scipy import optimize

#define the function we want to fit to
def test_func(x, a, b, c):
    return a+b*np.exp(-x/c)

params, params_covariance = optimize.curve_fit(test_func, ds.level.values, T_profile.values,
                                               p0=[2, 2, 2])

print(params)

In [ ]:
plt.figure(figsize=(10, 10))
T_profile.plot(y='level', yincrease=True, linewidth=5.0)
plt.plot(test_func(ds.level, *params),ds.level, 'g--', linewidth=5.0)
plt.gca().invert_yaxis()
plt.gca().tick_params('both',labelsize=25)
plt.ylabel('Depth (m)',fontsize=25)
plt.xlabel('Temperature ($^{\circ}C$)',fontsize=25);